- Automated generate test data

- https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/#choose-your-llm

In [3]:
from dotenv import load_dotenv
from ragas.testset import TestsetGenerator

In [2]:
load_dotenv("configs.env")

True